<div style="background-color: #f9f9fc; color: #333366; border-radius: 12px; margin: 20px auto; padding: 20px; border: 2px solid #ff4c4c; max-width: 1000px; font-family: Arial, sans-serif; line-height: 1.6;">
  <h2 style="text-align: center; color: #333366;">Time-Series Forecasting of Carbon Monoxide and Nitrogen Dioxide Levels</h2>

In [ ]:
!pip install numpy==1.23.5
print("M")

Build predictive models to forecast daily or hourly concentrations of **CO(GT)** and **NO₂(GT)** based on historical data.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore

# ML libraries
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# DL libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU

# Time Series Analysis libraries
from statsmodels.tsa.stattools import adfuller  # Augmented Dickey-Fuller (ADF) Test
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet

# Ignore warnings
import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('Time-Series_data.txt', delimiter=',')
df.head()

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
print(df.isnull().sum())

In [ ]:
df.shape

In [ ]:
df.nunique()

In [ ]:
df.duplicated().sum()

In [ ]:
# Step 1: Ensure numeric columns
numeric_columns = df.select_dtypes(include=['number'])  # Select only numeric columns

# Step 2: Check for negative values in each column
negative_counts = (numeric_columns < 0).sum()

# Step 3: Filter only columns with negative values
negative_counts = negative_counts[negative_counts > 0]

# Step 4: Display counts of negative values
print("Count of negative values in each column:")
print(negative_counts)

# Step 5: Visualization
import matplotlib.pyplot as plt

if not negative_counts.empty:
    plt.figure(figsize=(12, 6))
    negative_counts.plot(kind='bar', color='orange')
    plt.title('Count of Negative Values in Each Column')
    plt.xlabel('Columns')
    plt.ylabel('Count of Negative Values')
    plt.xticks(rotation=45)
    plt.show()
else:
    print("No negative values found in numeric columns.")

In [ ]:
#Converts the Time Date value to a string, last 4 characters of the string, representing the year.
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

In [ ]:
df.head()

In [ ]:
# Select only numeric columns
numeric_df = df.select_dtypes(include=['float64', 'int64'])

# Calculate the correlation matrix
correlation_matrix = numeric_df.corr()

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
plt.title("Correlation Heatmap")
plt.show()

<div style="background-color: #f9f9fc; color: #333366; border-radius: 12px; margin: 20px auto; padding: 20px; border: 2px solid #ff4c4c; max-width: 1000px; font-family: Arial, sans-serif; line-height: 1.6;">
  <h2 style="text-align: center; color: #333366;">Project Workflow</h2>

---
### **Deliverables**:
1. **Forecasting Models**:
   - Trained models for `CO(GT)` and `NO₂(GT)`.
2. **Forecast Visualization**:
   - Plots showing historical trends and future predictions.
3. **Evaluation Report**:
   - Metrics and comparison of different forecasting models.
4. **Insights and Recommendations**:
   - Suggestions for mitigating high levels of these gases based on predictions.


### **1. Data Preprocessing**
---
- **Date-Time Parsing**:
  - Combine the `Date` and `Time` columns into a single `datetime` column.
  - Set the `datetime` column as the index of the dataset.
- **Resampling**:
  - Aggregate the data into meaningful time intervals (e.g., hourly or daily averages).
- **Handle Missing Values**:
  - Use interpolation, mean, or advanced imputation techniques to fill missing data for `CO(GT)` and `NO₂(GT)`.
- **Outlier Detection**:
  - Remove or cap extreme values in `CO(GT)` and `NO₂(GT)` using statistical thresholds.

In [ ]:
# Date-Time Parsing
df['datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str))
df.drop(columns=['Date', 'Time'], inplace=True)
df.set_index('datetime', inplace=True)

In [ ]:
df.head()

In [ ]:
# Resample the data daily basis a. 
daily_data = df.resample('D').mean()
print(daily_data.head())

In [ ]:
# hourly_data = df.resample('h').mean()
# print(hourly_data.head())

### outliers 

In [ ]:
import matplotlib.pyplot as plt

# CO(GT) Boxplots
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
df['CO(GT)'].plot(kind='box', title='Boxplot of CO(GT)')
plt.ylabel('CO(GT)')

# NO2(GT) Boxplots
plt.subplot(1, 2, 2)
df['NO2(GT)'].plot(kind='box', title='Boxplot of NO2(GT)')
plt.ylabel('NO2(GT)')

plt.tight_layout()
plt.show()

In [1]:
# Function to cap outliers based on IQR
def cap_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)
    return df

# Cap outliers for CO(GT) and NO2(GT)
df = cap_outliers_iqr(df, 'CO(GT)')
df = cap_outliers_iqr(df, 'NO2(GT)')

NameError: name 'df' is not defined

In [ ]:
# Function to remove outliers based on IQR
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Remove outliers for CO(GT) and NO2(GT)
df = remove_outliers_iqr(df, 'CO(GT)')
df = remove_outliers_iqr(df, 'NO2(GT)')

# Reset index after removing outliers
df.reset_index(drop=True, inplace=True)

In [ ]:
from scipy.stats import zscore

# Z-Score Method
def detect_outliers_zscore(data, threshold=3):
    outliers = {}
    for col in data.select_dtypes(include=np.number):  # Numeric columns only
        z_scores = zscore(data[col].dropna())
        outliers[col] = data[col][np.abs(z_scores) > threshold]
    return outliers

# IQR Method
def detect_outliers_iqr(data):
    outliers = {}
    for col in data.select_dtypes(include=np.number):  # Numeric columns only
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers[col] = data[col][(data[col] < lower_bound) | (data[col] > upper_bound)]
    return outliers

# Detect outliers using both methods
zscore_outliers = detect_outliers_zscore(df)
iqr_outliers = detect_outliers_iqr(df)

# Print results
print("Outliers detected using Z-Score:")
for col, outlier_values in zscore_outliers.items():
    print(f"{col}: {len(outlier_values)} outliers")

print("\nOutliers detected using IQR:")
for col, outlier_values in iqr_outliers.items():
    print(f"{col}: {len(outlier_values)} outliers")

In [ ]:
# Remove outliers using Z-score method
def remove_outliers_zscore(data, threshold=3):
    for col in data.select_dtypes(include=np.number):
        z_scores = zscore(data[col].dropna())
        data = data[np.abs(z_scores) <= threshold]
    return data

# Remove outliers using IQR method
def remove_outliers_iqr(data):
    for col in data.select_dtypes(include=np.number):
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        data = data[(data[col] >= lower_bound) & (data[col] <= upper_bound)]
    return data

# Example usage
df_cleaned_zscore = remove_outliers_zscore(df.copy())
df_cleaned_iqr = remove_outliers_iqr(df.copy())

print("Dataset after removing outliers (Z-score):", df_cleaned_zscore.shape)
print("Dataset after removing outliers (IQR):", df_cleaned_iqr.shape)

In [ ]:
import matplotlib.pyplot as plt

# CO(GT) Boxplots
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
df['CO(GT)'].plot(kind='box', title='Boxplot of CO(GT)')
plt.ylabel('CO(GT)')

# NO2(GT) Boxplots
plt.subplot(1, 2, 2)
df['NO2(GT)'].plot(kind='box', title='Boxplot of NO2(GT)')
plt.ylabel('NO2(GT)')

plt.tight_layout()
plt.show()

In [ ]:
# Adjust Z-Score Threshold for Outlier Detection
z_threshold = 3.5  
df['CO(GT)_zscore'] = zscore(df['CO(GT)'])

# Identify Outliers
outliers = df.loc[abs(df['CO(GT)_zscore']) > z_threshold, 'CO(GT)']
print(f"Identified Outliers:\n{outliers}")

# Cap Outliers Instead of Replacing with Median
Q1 = df['CO(GT)'].quantile(0.25)
Q3 = df['CO(GT)'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Cap values at lower and upper bounds
df['CO(GT)'] = df['CO(GT)'].clip(lower=lower_bound, upper=upper_bound)

# Drop the temporary Z-score column
df.drop(columns=['CO(GT)_zscore'], inplace=True)

# Visualize the Results
# Boxplot
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
df['CO(GT)'].plot(kind='box', title='Boxplot of CO(GT) After Handling Outliers')
plt.subplot(1, 2, 2)
df['NO2(GT)'].plot(kind='box', title='Boxplot of NO2(GT)')
plt.tight_layout()
plt.show()

# Histogram for Distribution Check
plt.figure(figsize=(12, 6))
plt.hist(df['CO(GT)'], bins=30, edgecolor='k', alpha=0.7)
plt.title('Distribution of CO(GT) After Handling Outliers (Capping)')
plt.xlabel('CO(GT)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# CO(GT) and NO2(GT) Histplolt
plt.figure(figsize=(10, 5))

# CO(GT) Histplolt
plt.subplot(1, 2, 1)
df['CO(GT)'].plot(kind='hist', bins=30, title='Histogram of CO(GT)')
plt.xlabel('CO(GT)')

# NO2(GT) Histplolt
plt.subplot(1, 2, 2)
df['NO2(GT)'].plot(kind='hist', bins=30, title='Histogram of NO2(GT)')
plt.xlabel('NO2(GT)')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))

# CO(GT) Plot
plt.subplot(2, 1, 1)
plt.plot(df.index, df['CO(GT)'], 'o', alpha=0.5)
plt.title('Time Series of CO(GT)')
plt.ylabel('CO(GT)')

# NO2(GT) Plot
plt.subplot(2, 1, 2)
plt.plot(df.index, df['NO2(GT)'], 'o', alpha=0.5)
plt.title('Time Series of NO2(GT)')
plt.ylabel('NO2(GT)')

plt.tight_layout()
plt.show()

In [ ]:
df.columns

In [ ]:
import matplotlib.pyplot as plt

# Plotting CO(GT) historical data
plt.figure(figsize=(12, 6))
plt.plot(df['CO(GT)'], label='CO(GT)', color='blue')
plt.title('Historical Levels of CO(GT)')
plt.xlabel('Time')
plt.ylabel('Concentration')
plt.legend()
plt.show()

# Plotting NO2(GT) historical data
plt.figure(figsize=(12, 6))
plt.plot(df['NO2(GT)'], label='NO2(GT)', color='green')
plt.title('Historical Levels of NO2(GT)')
plt.xlabel('Time')
plt.ylabel('Concentration')
plt.legend()
plt.show()

<div style="background-color: #f9f9fc; color: #333366; border-radius: 12px; margin: 20px auto; padding: 20px; border: 2px solid #ff4c4c; max-width: 1000px; font-family: Arial, sans-serif; line-height: 1.6;">
  <h2 style="text-align: center; color: #333366;">2. Exploratory Data Analysis (EDA)</h2>

  ---
- **Trend Analysis**:
  - Visualize the long-term trends of `CO(GT)` and `NO₂(GT)`.
- **Seasonality**:
  - Identify seasonal patterns (e.g., daily or yearly fluctuations).
- **Correlation Analysis**:
  - Explore relationships between `CO(GT)`, `NO₂(GT)`, and other features (e.g., temperature or humidity).

# Performing trend analysis using the same dataset

In [ ]:
# Plot for CO(GT)
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['CO(GT)'], label='CO(GT)', color='blue', alpha=0.7)
plt.title('Long-Term Trend of CO(GT)')
plt.xlabel('Time')
plt.ylabel('Concentration')
plt.legend()
plt.show()

# Plot for NO2(GT)
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['NO2(GT)'], label='NO2(GT)', color='green', alpha=0.7)
plt.title('Long-Term Trend of NO2(GT)')
plt.xlabel('Time')
plt.ylabel('Concentration')
plt.legend()
plt.show()

In [ ]:
# Correlation between features
correlation = df[['CO(GT)', 'NO2(GT)', 'T', 'RH']].corr()

# Visualize correlation as a heatmap
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.heatmap(correlation, annot=True, cmap='coolwarm')
plt.title('Correlation Analysis')
plt.show()

In [ ]:
# long-term trends Analysis
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['CO(GT)'], label='CO(GT)', alpha=0.7)
plt.plot(df.index, df['NO2(GT)'], label='NO2(GT)', alpha=0.7)
plt.title('Long-Term Trends of CO(GT) and NO2(GT)')
plt.xlabel('Time')
plt.ylabel('Concentration')
plt.legend()
plt.show()

In [ ]:
# print(df['hour'].unique())  # Should show integers from 0 to 23
# print(hourly_mean.head())  # Should show grouped hourly means with index as hours

In [ ]:
plt.figure(figsize=(15, 8))
corr = df.select_dtypes(include=['number']).corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
# Ensure the index is in datetime format
df.index = pd.to_datetime(df.index)

# Extract the 'month' from the index
df['Month'] = df.index.month

# Group by month and calculate the average
monthly_avg = df.groupby('Month')[['CO(GT)', 'NO2(GT)']].mean()
print(monthly_avg)

# Step 4: Plot the seasonal trends
monthly_avg.plot(figsize=(12, 6), marker='o')
plt.title('Seasonality Analysis: Monthly Averages')
plt.xlabel('Month')
plt.ylabel('Average Concentration')
plt.legend(title='Pollutants')
plt.show()

In [ ]:
# Convert the index to datetime format
df.index = pd.to_datetime(df.index)

# Create the 'Hour' column
df['Hour'] = df.index.hour

# Calculate hourly averages
hourly_avg = df.groupby('Hour')[['CO(GT)', 'NO2(GT)']].mean()

# Display the hourly averages
print(hourly_avg)

# Step 5: Plot hourly averages
plt.figure(figsize=(12, 6))
plt.plot(hourly_avg.index, hourly_avg['CO(GT)'], label='CO(GT)', marker='o', color='blue')
plt.plot(hourly_avg.index, hourly_avg['NO2(GT)'], label='NO2(GT)', marker='o', color='green')
plt.title('Daily Seasonality Analysis')
plt.xlabel('Hour of Day')
plt.ylabel('Average Concentration')
plt.legend()
plt.grid()
plt.show()

In [ ]:
import seaborn as sns

# correlation matrix
correlation_matrix = df[['CO(GT)', 'NO2(GT)', 'T', 'RH']].corr()

# Plot heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Between Variables')
plt.show()

In [ ]:
# Pair plot to visualize relationships
sns.pairplot(df[['CO(GT)', 'NO2(GT)', 'T', 'RH']])
plt.show()

<div style="background-color: #f9f9fc; color: #333366; border-radius: 12px; margin: 20px auto; padding: 20px; border: 2px solid #ff4c4c; max-width: 1000px; font-family: Arial, sans-serif; line-height: 1.6;">
  <h2 style="text-align: center; color: #333366;">3. Feature Engineering</h2>
    
  ---
- Create lag features for `CO(GT)` and `NO₂(GT)` to incorporate historical values.
- Add time-based features (e.g., hour of the day, day of the week, month).
- Include environmental factors (`T`, `RH`, `AH`) as predictors.

In [ ]:
# Create lag features for CO(GT) and NO2(GT)
df['CO_lag1'] = df['CO(GT)'].shift(1)
df['CO_lag2'] = df['CO(GT)'].shift(2)

df['NO2_lag1'] = df['NO2(GT)'].shift(1)
df['NO2_lag2'] = df['NO2(GT)'].shift(2)

# Drop rows with NaN values caused by lagging
df = df.dropna()

In [ ]:
# Extract time-based features using .loc to modify the original DataFrame
df.loc[:, 'hour'] = df.index.hour
df.loc[:, 'day_of_week'] = df.index.dayofweek  # 0 = Monday, 6 = Sunday
df.loc[:, 'month'] = df.index.month

In [ ]:
# Ensure predictors are present
predictors = ['T', 'RH', 'AH', 'CO_lag1', 'CO_lag2', 'NO2_lag1', 'NO2_lag2', 'hour', 'day_of_week', 'month']

In [ ]:
# Define X (features) and y (target)
X = df[predictors]
y_CO = df['CO(GT)']
y_NO2 = df['NO2(GT)']

# Check the final feature set
print(X.head())

In [ ]:
# Display the first few rows to check lag features
print(df[['CO(GT)', 'CO_lag1', 'CO_lag2', 'NO2(GT)', 'NO2_lag1', 'NO2_lag2']].head(10))

In [ ]:
import matplotlib.pyplot as plt

# Plot CO(GT) with its lag features
plt.figure(figsize=(12, 6))
plt.plot(df['CO(GT)'], label='CO(GT)', color='blue')
plt.plot(df['CO_lag1'], label='CO_lag1', color='orange', linestyle='--')
plt.plot(df['CO_lag2'], label='CO_lag2', color='green', linestyle='--')
plt.title('CO(GT) and its Lag Features')
plt.xlabel('Time')
plt.ylabel('Concentration')
plt.legend()
plt.show()

# Plot NO2(GT) with its lag features
plt.figure(figsize=(12, 6))
plt.plot(df['NO2(GT)'], label='NO2(GT)', color='blue')
plt.plot(df['NO2_lag1'], label='NO2_lag1', color='orange', linestyle='--')
plt.plot(df['NO2_lag2'], label='NO2_lag2', color='green', linestyle='--')
plt.title('NO2(GT) and its Lag Features')
plt.xlabel('Time')
plt.ylabel('Concentration')
plt.legend()
plt.show()

<div style="background-color: #f9f9fc; color: #333366; border-radius: 12px; margin: 20px auto; padding: 20px; border: 2px solid #ff4c4c; max-width: 1000px; font-family: Arial, sans-serif; line-height: 1.6;">
  <h2 style="text-align: center; color: #333366;">4. Time-Series Forecasting</h2>


  ---
- Train separate models for **CO(GT)** and **NO₂(GT)**.
- Explore the following forecasting approaches:
  - **Statistical Models**:
    - Classical models like ARIMA or SARIMA for univariate forecasting.
  - **Machine Learning Models**:
    - Use regression models trained on lag features and external predictors.
  - **Deep Learning Models**:
    - Apply LSTM, GRU, or other RNN-based models for sequence forecasting.
- Evaluate and compare the models.

<div style="background-color: #f9f9fc; color: #333366; border-radius: 12px; margin: 20px auto; padding: 20px; border: 2px solid #ff4c4c; max-width: 1000px; font-family: Arial, sans-serif; line-height: 1.6;">
  <h2 style="text-align: center; color: #333366;">Steps for SARIMA Model Preparation and Implementation</h2>

---

### Step 1: Preparing the Dataset
- **Stationarity**: Ensure that the data is stationary by performing stationarity tests (e.g., ADF test).
- **Seasonality**: Properly capture seasonality and trends in the data.
- **Train-Test Split**: Split the dataset into training and testing sets.

### Step 2: Parameter Selection for SARIMA Model
- Identify the parameters \( p, d, q, P, D, Q, s \) for the SARIMA model.

### Step 3: Build the SARIMA Model
- Fit the SARIMA model using the parameters obtained (e.g., from Auto ARIMA).

### Step 4: Forecasting
- Generate separate forecasts for **CO** and **NO2** levels.

### Step 5: Evaluation
- Evaluate the performance of the forecasts using appropriate metrics (e.g., RMSE, MAE).

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Check stationarity for CO(GT)
result_co = adfuller(df['CO(GT)'])
print("CO(GT) - ADF Statistic:", result_co[0])
print("CO(GT) - p-value:", result_co[1])

# Check stationarity for NO2(GT)
result_no2 = adfuller(df['NO2(GT)'])
print("NO2(GT) - ADF Statistic:", result_no2[0])
print("NO2(GT) - p-value:", result_no2[1])


In [ ]:
# Differencing if needed
df['CO_diff'] = df['CO(GT)'].diff().dropna()
df['NO2_diff'] = df['NO2(GT)'].diff().dropna()

In [ ]:
from pmdarima import auto_arima

# Auto ARIMA for CO(GT)
auto_arima_co = auto_arima(df['CO(GT)'], seasonal=True, m=12)
print(auto_arima_co.summary())

# Auto ARIMA for NO2(GT)
auto_arima_no2 = auto_arima(df['NO2(GT)'], seasonal=True, m=12)
print(auto_arima_no2.summary())

In [ ]:
pip install --upgrade numpy scipy pmdarima

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Fit SARIMA model for CO(GT)
sarima_co = SARIMAX(df['CO(GT)'], 
                    order=(1, 1, 1), 
                    seasonal_order=(1, 1, 1, 12))
sarima_co_fit = sarima_co.fit()
print(sarima_co_fit.summary())

In [ ]:
# Fit SARIMA model for NO2(GT)
sarima_no2 = SARIMAX(df['NO2(GT)'], 
                     order=(1, 1, 1), 
                     seasonal_order=(1, 1, 1, 12))
sarima_no2_fit = sarima_no2.fit()
print(sarima_no2_fit.summary())

In [ ]:
forecast_co = sarima_co_fit.get_forecast(steps=30)
forecast_co_values = forecast_co.predicted_mean
forecast_co_ci = forecast_co.conf_int()

# Plot CO(GT) Forecast
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(df['CO(GT)'], label='Observed CO(GT)')
plt.plot(forecast_co_values, label='Forecasted CO(GT)', color='red')
plt.fill_between(forecast_co_values.index, 
                 forecast_co_ci.iloc[:, 0], 
                 forecast_co_ci.iloc[:, 1], color='pink', alpha=0.3)
plt.legend()
plt.title('CO(GT) Forecast')
plt.show()

In [ ]:
forecast_no2 = sarima_no2_fit.get_forecast(steps=30)
forecast_no2_values = forecast_no2.predicted_mean
forecast_no2_ci = forecast_no2.conf_int()

# Plot NO2(GT) Forecast
plt.figure(figsize=(12, 6))
plt.plot(df['NO2(GT)'], label='Observed NO2(GT)')
plt.plot(forecast_no2_values, label='Forecasted NO2(GT)', color='red')
plt.fill_between(forecast_no2_values.index, 
                 forecast_no2_ci.iloc[:, 0], 
                 forecast_no2_ci.iloc[:, 1], color='pink', alpha=0.3)
plt.legend()
plt.title('NO2(GT) Forecast')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error

# RMSE for CO(GT)
rmse_co = mean_squared_error(df['CO(GT)'].iloc[-30:], forecast_co_values, squared=False)
print("RMSE for CO(GT):", rmse_co)

# RMSE for NO2(GT)
rmse_no2 = mean_squared_error(df['NO2(GT)'].iloc[-30:], forecast_no2_values, squared=False)
print("RMSE for NO2(GT):", rmse_no2)

<div style="background-color: #f9f9fc; color: #333366; border-radius: 12px; margin: 20px auto; padding: 20px; border: 2px solid #ff4c4c; max-width: 1000px; font-family: Arial, sans-serif; line-height: 1.6;">
  <h2 style="text-align: center; color: #333366;">Steps to Build LSTM Models for CO(GT) and NO2(GT)</h2>


---

## Step 1: Import Libraries

## Step 2: Data Preparation

## Step 3: Create Time Series Data
- Create sequences for training and testing.
    - **Function to create sequences for LSTM**
    - **Create sequences for CO(GT)**
    - **Create sequences for NO2(GT)**
    - **Reshape inputs for LSTM**

## Step 4: Train-Test Split
- Split data into training and testing sets.
    - **Split data for CO(GT)**
    - **Split data for NO2(GT)**

## Step 5: Build LSTM Models
### For CO(GT):
- **LSTM model for CO(GT)**
- **Train the model**

### For NO2(GT):
- **LSTM model for NO2(GT)**
- **Train the model**

## Step 6: Evaluate Models
- Plot training and validation loss.
    - **Plot for CO(GT)**
    - **Plot for NO2(GT)**

## Step 7: Make Predictions
- Predict using the trained models and inverse transform to get the actual values.

### For CO(GT):
- **Predict on test data**
- **Plot CO(GT) Predictions**

### For NO2(GT):
- **Predict on test data**
- **Plot NO2(GT) Predictions**

In [ ]:
# Select columns for CO(GT) and NO2(GT)
data_co = df[['CO(GT)']].dropna()
data_no2 = df[['NO2(GT)']].dropna()

# Normalize the data
scaler_co = MinMaxScaler(feature_range=(0, 1))
scaled_co = scaler_co.fit_transform(data_co)

scaler_no2 = MinMaxScaler(feature_range=(0, 1))
scaled_no2 = scaler_no2.fit_transform(data_no2)

In [ ]:
# Function to create sequences for LSTM
def create_sequences(data, time_steps=30):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps, 0])
        y.append(data[i+time_steps, 0])
    return np.array(X), np.array(y)

# Create sequences for CO(GT)
time_steps = 30
X_co, y_co = create_sequences(scaled_co, time_steps)

# Create sequences for NO2(GT)
X_no2, y_no2 = create_sequences(scaled_no2, time_steps)

# Reshape inputs for LSTM
X_co = X_co.reshape((X_co.shape[0], X_co.shape[1], 1))
X_no2 = X_no2.reshape((X_no2.shape[0], X_no2.shape[1], 1))

In [ ]:
# Split data for CO(GT)
train_size_co = int(len(X_co) * 0.8)
X_train_co, X_test_co = X_co[:train_size_co], X_co[train_size_co:]
y_train_co, y_test_co = y_co[:train_size_co], y_co[train_size_co:]

# Split data for NO2(GT)
train_size_no2 = int(len(X_no2) * 0.8)
X_train_no2, X_test_no2 = X_no2[:train_size_no2], X_no2[train_size_no2:]
y_train_no2, y_test_no2 = y_no2[:train_size_no2], y_no2[train_size_no2:]

In [ ]:
# LSTM model for CO(GT)
model_co = Sequential([
    LSTM(50, return_sequences=True, input_shape=(time_steps, 1)),
    LSTM(50),
    Dense(1)
])

model_co.compile(optimizer='adam', loss='mse')
# Train the model
history_co = model_co.fit(X_train_co, y_train_co, epochs=20, batch_size=32, validation_data=(X_test_co, y_test_co), verbose=1)

In [ ]:
# LSTM model for NO2(GT)
model_no2 = Sequential([
    LSTM(50, return_sequences=True, input_shape=(time_steps, 1)),
    LSTM(50),
    Dense(1)
])

model_no2.compile(optimizer='adam', loss='mse')
# Train the model
history_no2 = model_no2.fit(X_train_no2, y_train_no2, epochs=20, batch_size=32, validation_data=(X_test_no2, y_test_no2), verbose=1)

In [ ]:
# Plot for CO(GT)
plt.figure(figsize=(12, 6))
plt.plot(history_co.history['loss'], label='Training Loss - CO(GT)')
plt.plot(history_co.history['val_loss'], label='Validation Loss - CO(GT)')
plt.title('Training and Validation Loss for CO(GT)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot for NO2(GT)
plt.figure(figsize=(12, 6))
plt.plot(history_no2.history['loss'], label='Training Loss - NO2(GT)')
plt.plot(history_no2.history['val_loss'], label='Validation Loss - NO2(GT)')
plt.title('Training and Validation Loss for NO2(GT)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Predict on test data
predicted_co = model_co.predict(X_test_co)
predicted_co = scaler_co.inverse_transform(predicted_co.reshape(-1, 1))
y_test_co_actual = scaler_co.inverse_transform(y_test_co.reshape(-1, 1))

# Plot CO(GT) Predictions
plt.figure(figsize=(12, 6))
plt.plot(y_test_co_actual, label='Actual CO(GT)')
plt.plot(predicted_co, label='Predicted CO(GT)')
plt.title('Actual vs Predicted CO(GT)')
plt.legend()
plt.show()

In [ ]:
# Predict on test data
predicted_no2 = model_no2.predict(X_test_no2)
predicted_no2 = scaler_no2.inverse_transform(predicted_no2.reshape(-1, 1))
y_test_no2_actual = scaler_no2.inverse_transform(y_test_no2.reshape(-1, 1))

# Plot NO2(GT) Predictions
plt.figure(figsize=(12, 6))
plt.plot(y_test_no2_actual, label='Actual NO2(GT)')
plt.plot(predicted_no2, label='Predicted NO2(GT)')
plt.title('Actual vs Predicted NO2(GT)')
plt.legend()
plt.show()

<div style="background-color: #f9f9fc; color: #333366; border-radius: 12px; margin: 20px auto; padding: 20px; border: 2px solid #ff4c4c; max-width: 1000px; font-family: Arial, sans-serif; line-height: 1.6;">
  <h2 style="text-align: center; color: #333366;">Build GRU models for predicting CO(GT) and NO2(GT)</h2>

In [ ]:
# Select columns for CO(GT) and NO2(GT)
data_co = df[['CO(GT)']].dropna()
data_no2 = df[['NO2(GT)']].dropna()

# Normalize the data
scaler_co = MinMaxScaler(feature_range=(0, 1))
scaled_co = scaler_co.fit_transform(data_co)

scaler_no2 = MinMaxScaler(feature_range=(0, 1))
scaled_no2 = scaler_no2.fit_transform(data_no2)

In [ ]:
# Function to create sequences for GRU
def create_sequences(data, time_steps=30):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps, 0])
        y.append(data[i+time_steps, 0])
    return np.array(X), np.array(y)

# Create sequences for CO(GT)
time_steps = 30
X_co, y_co = create_sequences(scaled_co, time_steps)

# Create sequences for NO2(GT)
X_no2, y_no2 = create_sequences(scaled_no2, time_steps)

# Reshape inputs for GRU
X_co = X_co.reshape((X_co.shape[0], X_co.shape[1], 1))
X_no2 = X_no2.reshape((X_no2.shape[0], X_no2.shape[1], 1))

In [ ]:
# Split data for CO(GT)
train_size_co = int(len(X_co) * 0.8)
X_train_co, X_test_co = X_co[:train_size_co], X_co[train_size_co:]
y_train_co, y_test_co = y_co[:train_size_co], y_co[train_size_co:]

# Split data for NO2(GT)
train_size_no2 = int(len(X_no2) * 0.8)
X_train_no2, X_test_no2 = X_no2[:train_size_no2], X_no2[train_size_no2:]
y_train_no2, y_test_no2 = y_no2[:train_size_no2], y_no2[train_size_no2:]

In [ ]:
# GRU model for CO(GT)
model_co = Sequential([
    GRU(50, return_sequences=True, input_shape=(time_steps, 1)),
    GRU(50),
    Dense(1)
])

model_co.compile(optimizer='adam', loss='mse')
# Train the model
history_co = model_co.fit(X_train_co, y_train_co, epochs=20, batch_size=32, validation_data=(X_test_co, y_test_co), verbose=1)

In [ ]:
# GRU model for NO2(GT)
model_no2 = Sequential([
    GRU(50, return_sequences=True, input_shape=(time_steps, 1)),
    GRU(50),
    Dense(1)
])

model_no2.compile(optimizer='adam', loss='mse')
# Train the model
history_no2 = model_no2.fit(X_train_no2, y_train_no2, epochs=20, batch_size=32, validation_data=(X_test_no2, y_test_no2), verbose=1)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(history_co.history['loss'], label='Training Loss - CO(GT)')
plt.plot(history_co.history['val_loss'], label='Validation Loss - CO(GT)')
plt.title('Training and Validation Loss for CO(GT)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(history_no2.history['loss'], label='Training Loss - NO2(GT)')
plt.plot(history_no2.history['val_loss'], label='Validation Loss - NO2(GT)')
plt.title('Training and Validation Loss for NO2(GT)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

### For CO(GT)

In [ ]:
# Predict on test data
predicted_co = model_co.predict(X_test_co)
predicted_co = scaler_co.inverse_transform(predicted_co.reshape(-1, 1))
y_test_co_actual = scaler_co.inverse_transform(y_test_co.reshape(-1, 1))

# Plot CO(GT) Predictions
plt.figure(figsize=(12, 6))
plt.plot(y_test_co_actual, label='Actual CO(GT)')
plt.plot(predicted_co, label='Predicted CO(GT)')
plt.title('Actual vs Predicted CO(GT)')
plt.legend()
plt.show()

### For NO2(GT)

In [ ]:
# Predict on test data
predicted_no2 = model_no2.predict(X_test_no2)
predicted_no2 = scaler_no2.inverse_transform(predicted_no2.reshape(-1, 1))
y_test_no2_actual = scaler_no2.inverse_transform(y_test_no2.reshape(-1, 1))

# Plot NO2(GT) Predictions
plt.figure(figsize=(12, 6))
plt.plot(y_test_no2_actual, label='Actual NO2(GT)')
plt.plot(predicted_no2, label='Predicted NO2(GT)')
plt.title('Actual vs Predicted NO2(GT)')
plt.legend()
plt.show()

In [ ]:
print(df.columns)

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Check for stationarity in the 'CO(GT)' column
result = adfuller(df['CO(GT)'])
print("ADF Statistic:", result[0])
print("p-value:", result[1])

if result[1] > 0.05:
    print("The series is not stationary. Differencing will be applied.")
    df['CO(GT)_Diff'] = df['CO(GT)'].diff().dropna()
else:
    print("The series is stationary. Proceeding without differencing.")


In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Fit the ARIMA model
model = ARIMA(df['CO(GT)'], order=(1, 1, 1))  # Replace with the correct column name
arima_fitted = model.fit()

# Forecast future values
forecast_steps = 20
forecast = arima_fitted.forecast(steps=forecast_steps)

# Print the forecast
print("ARIMA Forecast:\n", forecast)

In [ ]:
# Plot the forecast
import matplotlib.pyplot as plt

# Replace 'CO(GT)' with your actual column name
plt.figure(figsize=(12, 6))
plt.plot(df['CO(GT)'], label='Observed')  # Observed data
plt.plot(range(len(df), len(df) + forecast_steps), forecast, label='Forecast', color='red')  # Forecasted data
plt.legend()
plt.title('ARIMA Forecast for CO(GT)')
plt.show()

In [ ]:
# Assign co_series to a column in the DataFrame
co_series = df['CO(GT)']  # Replace 'CO(GT)' with your desired column name

# Check the first few rows and total number of data points
print(co_series.head())  # First few rows
print(len(co_series))    # Total number of data points

# Check for missing values and handle them
print("Number of missing values:", co_series.isnull().sum())  # Check NaNs
co_series = co_series.dropna()  # Drop NaN values

# Verify the updated series
print("Number of data points after dropping NaNs:", len(co_series))

In [ ]:
co_series_diff = co_series.diff().dropna()
print(co_series_diff.head())  # Ensure it has data
print(len(co_series_diff))  # Check the number of data points

#### Machine Learning Models (Lag Features)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Simulate or use your dataset
timestamps = pd.date_range(start='2023-01-01', periods=100, freq='D')
df = pd.DataFrame({'CO(GT)': np.random.rand(100) * 10}, index=timestamps)

# Create lag features
df['CO_lag1'] = df['CO(GT)'].shift(1)

# Drop rows with NaN values
ml_data = df.dropna().copy()

# Ensure there is enough data
if ml_data.empty or len(ml_data) < 5:
    print("Not enough data points after creating lag features. Using simulated data for testing.")
    timestamps = pd.date_range(start='2023-01-01', periods=50, freq='D')
    df = pd.DataFrame({'CO(GT)': np.random.rand(50) * 10}, index=timestamps)
    df['CO_lag1'] = df['CO(GT)'].shift(1)
    ml_data = df.dropna().copy()

# Define feature matrix X and target variable y
X = ml_data[['CO_lag1']]
y = ml_data['CO(GT)']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Random Forest - MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}")

# Visualize actual vs predicted values
plt.figure(figsize=(10, 6))
plt.plot(y_test.values, label='Actual Values', color='blue')
plt.plot(y_pred, label='Predicted Values', color='red')
plt.title('Random Forest: Actual vs Predicted Values')
plt.xlabel('Index')
plt.ylabel('CO(GT)')
plt.legend()
plt.show()

### Machine Learning Models

<div style="background-color: #f9f9fc; color: #333366; border-radius: 12px; margin: 20px auto; padding: 20px; border: 2px solid #ff4c4c; max-width: 1000px; font-family: Arial, sans-serif; line-height: 1.6;">
  <h2 style="text-align: center; color: #333366;">5. Model Evaluation</h2>

  ---
- Use appropriate metrics for forecasting:
  - Mean Absolute Error (MAE)
  - Root Mean Squared Error (RMSE)
  - Mean Absolute Percentage Error (MAPE)
- Plot actual vs. predicted values to visualize performance.

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Calculate predictions for CO(GT)
y_pred_co = model_co.predict(X_test_co)

# Convert predictions and actual values back to original scale if scaled
# If data was scaled with MinMaxScaler, use scaler.inverse_transform
y_pred_co_original = y_pred_co
y_test_co_original = y_test_co

# Metrics for CO(GT)
mae_co = mean_absolute_error(y_test_co_original, y_pred_co_original)
rmse_co = np.sqrt(mean_squared_error(y_test_co_original, y_pred_co_original))
r2_co = r2_score(y_test_co_original, y_pred_co_original)
mape_co = np.mean(np.abs((y_test_co_original - y_pred_co_original) / y_test_co_original)) * 100

print("CO(GT) - Metrics")
print(f"MAE: {mae_co}")
print(f"RMSE: {rmse_co}")
print(f"R²: {r2_co}")
print(f"MAPE: {mape_co}%")

# Calculate predictions for NO2(GT)
y_pred_no2 = model_no2.predict(X_test_no2)

# Convert predictions and actual values back to original scale if scaled
# If data was scaled with MinMaxScaler, use scaler.inverse_transform
y_pred_no2_original = y_pred_no2
y_test_no2_original = y_test_no2

# Metrics for NO2(GT)
mae_no2 = mean_absolute_error(y_test_no2_original, y_pred_no2_original)
rmse_no2 = np.sqrt(mean_squared_error(y_test_no2_original, y_pred_no2_original))
r2_no2 = r2_score(y_test_no2_original, y_pred_no2_original)
mape_no2 = np.mean(np.abs((y_test_no2_original - y_pred_no2_original) / y_test_no2_original)) * 100

print("\nNO2(GT) - Metrics")
print(f"MAE: {mae_no2}")
print(f"RMSE: {rmse_no2}")
print(f"R²: {r2_no2}")
print(f"MAPE: {mape_no2}%")

<div style="background-color: #f9f9fc; color: #333366; border-radius: 12px; margin: 20px auto; padding: 20px; border: 2px solid #ff4c4c; max-width: 1000px; font-family: Arial, sans-serif; line-height: 1.6;">
  <h2 style="text-align: center; color: #333366;">Plot Actual vs. Predicted for CO(GT)>

In [ ]:
import matplotlib.pyplot as plt

# Plot Actual vs. Predicted for CO(GT)
plt.figure(figsize=(10, 6))
plt.plot(y_test_co_original, label="Actual Values", color="blue")
plt.plot(y_pred_co_original, label="Predicted Values", color="red")
plt.title("Actual vs Predicted Values for CO(GT)")
plt.xlabel("Time Steps")
plt.ylabel("CO(GT) Values")
plt.legend()
plt.grid()
plt.show()

# Plot Actual vs. Predicted for NO2(GT)

In [ ]:
# Plot Actual vs. Predicted for NO2(GT)
plt.figure(figsize=(10, 6))
plt.plot(y_test_no2_original, label="Actual Values", color="blue")
plt.plot(y_pred_no2_original, label="Predicted Values", color="red")
plt.title("Actual vs Predicted Values for NO2(GT)")
plt.xlabel("Time Steps")
plt.ylabel("NO2(GT) Values")
plt.legend()
plt.grid()
plt.show()

# Both plot

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(12, 10))

# Plot for CO(GT)
axs[0].plot(y_test_co_original, label="Actual Values (CO)", color="blue")
axs[0].plot(y_pred_co_original, label="Predicted Values (CO)", color="red")
axs[0].set_title("Actual vs Predicted Values for CO(GT)")
axs[0].set_xlabel("Time Steps")
axs[0].set_ylabel("CO(GT) Values")
axs[0].legend()
axs[0].grid()

# Plot for NO2(GT)
axs[1].plot(y_test_no2_original, label="Actual Values (NO2)", color="blue")
axs[1].plot(y_pred_no2_original, label="Predicted Values (NO2)", color="red")
axs[1].set_title("Actual vs Predicted Values for NO2(GT)")
axs[1].set_xlabel("Time Steps")
axs[1].set_ylabel("NO2(GT) Values")
axs[1].legend()
axs[1].grid()

# Adjust layout
plt.tight_layout()
plt.show()

<div style="background-color: #f9f9fc; color: #333366; border-radius: 12px; margin: 20px auto; padding: 20px; border: 2px solid #ff4c4c; max-width: 1000px; font-family: Arial, sans-serif; line-height: 1.6;">
  <h2 style="text-align: center; color: #333366;">6. Visualization and Insights</h2>

  ---
- Generate forecasts for future dates and visualize the trends.
- Highlight seasonal patterns, peak times, and periods of concern.
- Provide actionable recommendations based on predicted trends.

In [ ]:
# Generate future forecasts
future_steps = 50  # Number of steps to forecast into the future
future_forecast_co = model_co.predict(X_test_co[-future_steps:])
future_forecast_no2 = model_no2.predict(X_test_no2[-future_steps:])

# Visualize forecasts
plt.figure(figsize=(12, 6))
plt.plot(range(len(y_test_co_original)), y_test_co_original, label='Actual Values (CO)', color='blue')
plt.plot(range(len(y_test_co_original), len(y_test_co_original) + future_steps),
         future_forecast_co, label='Forecasted Values (CO)', color='red')
plt.title('Future Forecast for CO(GT)')
plt.xlabel('Time Steps')
plt.ylabel('CO(GT) Values')
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(range(len(y_test_no2_original)), y_test_no2_original, label='Actual Values (NO2)', color='blue')
plt.plot(range(len(y_test_no2_original), len(y_test_no2_original) + future_steps),
         future_forecast_no2, label='Forecasted Values (NO2)', color='red')
plt.title('Future Forecast for NO2(GT)')
plt.xlabel('Time Steps')
plt.ylabel('NO2(GT) Values')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Check available columns
print("Available columns in the DataFrame:")
print(df.columns)

# Dynamically include 'NO2(GT)' only if it exists
columns_to_analyze = ['CO(GT)']
if 'NO2(GT)' in df.columns:
    columns_to_analyze.append('NO2(GT)')

# Group by hour
hourly_avg = df.groupby(df.index.hour)[columns_to_analyze].mean()

# Plot hourly patterns
plt.figure(figsize=(12, 6))
for column in columns_to_analyze:
    plt.plot(hourly_avg.index, hourly_avg[column], label=column, marker='o')

plt.title('Hourly Seasonal Patterns')
plt.xlabel('Hour of Day')
plt.ylabel('Average Concentration')
plt.legend()
plt.grid()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Check available column names
print("Available columns in the DataFrame:")
print(df.columns)

# Verify if 'NO2(GT)' exists in the DataFrame
if 'NO2(GT)' not in df.columns:
    print("Column 'NO2(GT)' not found. Please check the column name.")
    # Use 'CO(GT)' only if 'NO2(GT)' is missing
    hourly_avg = df.groupby(df.index.hour)[['CO(GT)']].mean()
    
    # Plot for 'CO(GT)' only
    plt.figure(figsize=(12, 6))
    plt.plot(hourly_avg.index, hourly_avg['CO(GT)'], label='CO(GT)', marker='o')
    plt.title('Seasonal Patterns: Hourly Averages for CO(GT)')
    plt.xlabel('Hour of the Day')
    plt.ylabel('Average Concentration')
    plt.legend()
    plt.grid()
    plt.show()
else:
    # Proceed if 'NO2(GT)' exists
    hourly_avg = df.groupby(df.index.hour)[['CO(GT)', 'NO2(GT)']].mean()

    # Plot for both 'CO(GT)' and 'NO2(GT)'
    plt.figure(figsize=(12, 6))
    plt.plot(hourly_avg.index, hourly_avg['CO(GT)'], label='CO(GT)', marker='o', color='blue')
    plt.plot(hourly_avg.index, hourly_avg['NO2(GT)'], label='NO2(GT)', marker='o', color='green')
    plt.title('Seasonal Patterns: Hourly Averages')
    plt.xlabel('Hour of the Day')
    plt.ylabel('Average Concentration')
    plt.legend()
    plt.grid()
    plt.show()

<div style="background-color: #f9f9fc; color: #333366; border-radius: 12px; margin: 20px auto; padding: 20px; border: 2px solid #ff4c4c; max-width: 1000px; font-family: Arial, sans-serif; line-height: 1.6;">
  <h2 style="text-align: center; color: #333366;">7. Provide Actionable Recommendations Based on Predicted Trends</h2>

  ---

#### Steps:
- Based on forecasted trends and seasonal patterns, suggest actionable steps.
- Recommendations could include:
  - **Policy-level actions**: Stricter emissions control during peak times.
  - **Public advisories**: Alerts for high-pollution periods.
  - **Infrastructure planning**: Adjusting traffic or industry schedules.

---

#### Example Recommendations:

##### **For Policymakers:**
- Implement traffic restrictions during peak pollution hours to reduce **CO(GT)** levels.
- Strengthen industrial emissions regulations during winter months to control **NO2(GT)**.

##### **For the Public:**
- Avoid outdoor activities during peak pollution hours.
- Use masks or air purifiers in high-pollution periods.

##### **For City Planners:**
- Increase green spaces in high-pollution areas.
- Optimize traffic flow during peak times to reduce congestion.